# 优化器

机器学习五个步骤：数据->模型->损失->优化器->迭代训练

前向传播得到模型输出与真实标签的差异，称之为**损失**。

通过损失，进入反向传播得到参数的梯度、

接下去**优化器根据梯度来更新参数，使得损失不断地减低**。

## 概念

PyTorch优化器：**管理并更新**模型中可学习参数的值，使得模型输出更接近真实标签。

更新参数一般采用梯度下降：
- 导数：函数在指定坐标轴上的变化率
- 方向导数：指定方向上的变化率
- 梯度：一个向量，方向为方向导数取得最大值的方向（增长最快的方向）

梯度下降：沿着梯度的负方向去变化，这样函数的下降也是最快的。往往采用梯度下降的方式去更新权值，使得函数的下降尽量的快。

## Optimizer基本属性与方法

基本属性：

```python
class Optimizer(object):
    def __init__(self, params, defaults):
        self.defaults = defaults
        self.state = defaultdict(dict)
        self.param_groups = []
        
        param_groups = [{'params': param_groups}]
```

- defaults：优化器超参数，存储学习率、momentum值、衰减系数等
- state：参数缓存，如momentum缓存（使用前几次梯度进行平均）
- param_groups：管理的参数组，一个列表，每个元素是一个字典，字典中有key，key里的值才是我们真正的参数（进行参数管理）
- \_step_count：记录更新次数，学习率中调整使用，比如迭代100次后更新学习率，记录这里的100

基本方法：

```python
class Optimizer(object):
    def zero_grad(self):
        for group in self.param_groups:
            for p in group['params']:
                if p.grad is not None:
                    p.grad.detach_()
                    p.grad.zero_()
```

- zero_grad()：清空所有管理参数的梯度（PyTorch张量不自动清零）
- step()：执行一步更新
- add_param_group()：添加参数组（优化器管理很多参数，这些参数可以分组），对不同组的参数设置不同的超参数，比如模型微调中，希望前面特征提取的那些层学习率小一些，后面新加的层学习率大一些更新快一些，就可以用这个方法。
```python
class Optimizer(object):
    def add_param_group(self, param_group):
        for group in self.param_groups:
            param_set.update(set(group['params']))
        
        self.param_groups.append(param_group)
```
- state_dict()：获取优化器当前状态信息字典
- load_state_dict()：加载状态信息字典，这两个方法用于模型断点的一个续训练，一般模型在训练时，经历若干个epoch之后就要保存当前的状态信息。
```python
class Optimizer(object):
    def state_dict(self):
        return {
            'state': packed_state,
            'param_groups': param_groups
         }
    
    def load_state_dict(self, state_dict):
```

进行优化器运行机制的调试：

<img style="float: center;" src="images/99.png" width="70%">

进入sgd.py的SGD类，首先对各参数进行判断：
<img style="float: center;" src="images/100.png" width="70%">

SGD类继承于optimizer，因此将代码运行到父类初始化这一行，点击步入：
<img style="float: center;" src="images/101.png" width="70%">

这里是optimizer的\_\_init\_\_初始化部分，可以看到上面介绍的各个属性以及他们的方法，此处有一个最重要的就是参数组的添加：
<img style="float: center;" src="images/102.png" width="70%">

初始化SGD的时候传入一个形参optim.SGD(net.parameters(), lr=LR, momentum=0.9)，这里net.parameter()就是神经网络的每层参数，SGD在初始化时会把这些参数以参数组的方式存起来，上图中的params就是神经网络每一层的参数。

当执行完初始化参数时：
<img style="float: center;" src="images/103.png" width="70%">

这就是优化器的初始工作了，初始化完之后，进行梯度清空，然后更新梯度：
<img style="float: center;" src="images/104.png" width="70%">

优化器具体方法：

1. step()：一次梯度下降更新参数
<img style="float: center;" src="images/105.png" width="70%">

2. zero_grad()：将梯度清零
<img style="float: center;" src="images/106.png" width="70%">

3. add_param_group()：添加参数组（在迁移学习中非常实用）
<img style="float: center;" src="images/107.png" width="70%">

4. state_dict()和load_state_dict()：用于保存和加载优化器状态信息，通常用作断点续训练（例如训练一个模型，训练了10次停电了，如果不保存的话就需要从头开始训练，但如果有这两个方法，就可以接着上次训练的地方再次训练）

首先是state_dict()
<img style="float: center;" src="images/108.png" width="70%">

可以看到state_dict()保存优化器的各种状态信息，通过torch.save可以保存这些状态到文件(.pkl)，之后通过load_state_dict()导入这个状态信息，让优化器在这个基础上继续训练：
<img style="float: center;" src="images/109.png" width="70%">

以上就是优化器的初始化和优化器的5各方法的使用，了解这些知识后，就知道优化器的运行机制，管理和更新模型的可学习参数（管理是通过各种属性（尤其是param_groups），更新通过各种方法（主要是step()方法））

## 常用优化器

### 学习率

梯度下降过程中，学习率用于控制参数更新的步伐作用：$w_{i+1}=w_i-LR\times grad(w_i)$

如果没有学习率LR，往往可能由于梯度过大而错过最优值，例如下面这种状况：
<img style="float: center;" src="images/110.png" width="40%">

随着迭代次数的增加，反而越增越大，因为这个步子太大了，跳过了最优值。

因此需要一个参数（学习率）来控制这个跨度，如果不控制，可以发现loss是不断上升的，说明这个跨度是有问题的：
<img style="float: center;" src="images/111.png" width="70%">

尝试改小一点学习率：
<img style="float: center;" src="images/112.png" width="70%">

当loss只上升不降的时候，可能是学习率的问题，通常可以尝试一个小的学习率，慢慢去进行优化。

学习率一般是需要调的一个非常重要的超参数，一般是给定一个范围，然后画出loss的变化，看看哪个学习率比较好：
<img style="float: center;" src="images/113.png" width="70%">

### 动量

Momentum：结合当前梯度与上一次更新信息，用于当前更新。

举个例子：
<img style="float: center;" src="images/114.png" width="70%">

动量如何作用于更新？

【指数加权平均】：时间序列中用于求取平均值，要求当前时刻的平均值，距离当前时刻越近的参数值的参考性越大，所占权重越大，这个权重随时间间隔的增大呈指数下降，所以叫指数加权平均。

$v_t=\beta * v_{t-1}+(1-\beta)*\theta_t$

其中，$v_t$是当前时刻的一个平均值，该值由两项构成：
- 当前时刻的参数值$\theta_t$，权重是$1-\beta$
- 上一时刻的平均值，权重是$\beta$

看下图温度图像，横轴是天数，纵轴是温度：
<img style="float: center;" src="images/115.png" width="70%">
假设想求第100天温度的一个平均值，则根据上述公式：

$v_{100}=\beta * v_{99}+(1-\beta)*\theta_{100}$

$=(1-\beta) * \theta_{100}+\beta * (\beta * v_{98} + (1-\beta)*\theta_{99})$

$=(1-\beta) * \theta_{100}+ (1-\beta) * \beta * \theta_{99} + (\beta^2 * v_{98})$

$=(1-\beta) * \theta_{100}+ (1-\beta) * \beta * \theta_{99} + (1-\beta)*\beta^2 * \theta_{98} +(\beta^3 * v_{97})$

$=(1-\beta) * \beta^0 * \theta_{100}+ (1-\beta) * \beta^1 * \theta_{99} + (1-\beta)*\beta^2 * \theta_{98} +(\beta^3 * v_{97})$

$=\sum^N_i(1-\beta) * \beta^i * \theta_{N-i}$

可以发现，距离当前时刻越远的那些$\theta$值，它的权重越来越小，因为$\beta\lt 1$，呈指数下降。

可以观察$(1-\beta)*\beta^i$如何变化：
- 距离当前时刻越远，对当前时刻的平均值影响就越小
- 距离当前时刻越近，对当前时刻的平均值影响就越大
- 这就是指数加权平均的思想
<img style="float: center;" src="images/116.png" width="70%">

观察$\beta$的值的变化：
<img style="float: center;" src="images/117.png" width="70%">

可以发现，当$\beta$值越小，它关注前面一段时刻的距离就越短（$\beta$控制记忆周期的长短，或者平均过去多少天的数据），这个天数是$\frac{1}{1-\beta}$，通常$\beta$设置为0.9，即关注过去10天左右的一个温度。
<img style="float: center;" src="images/118.png" width="70%">

上图是不同$\beta$下得到的一个温度变化曲线：
- 红色：$\beta=0.9$，即过去10天温度的平均值
- 绿色：$\beta=0.98$，即过去50天温度的平均值
- 黄色：$\beta=0.5$，即过去2天温度的平均值

可以发现，当$\beta$值越高，最终得到的温度变化曲线会平缓一些，因为多平均了几天的温度，缺点是曲线进一步右移，因为现在平均的温度值更多，要平均更多的值，指数加权平均公式，在温度变化时，适应的更缓慢一些，所以出现延迟。而如果$\beta$值越低，则平均数据太少，曲线会有很大的噪声，更有可能出现异常值，但这个曲线可以更快速适应温度的变化。

一般取$\beta=0.9$

Momentum梯度下降：计算梯度的指数加权平均数，并利用该梯度更新权重。
- 普通的梯度下降：$w_{i+1}=w_i-lr*g(w_i)$
- Momentum梯度下降：$v_i=m*v_{i-1}+g(w_i), w_{i+1}=w_i-lr*v_i$

此处$m$就是momentum系数，$v_i$表示更新量（即考虑当前梯度，也考虑上一次梯度的更新信息处），$g(w_i)$是$w_i$的梯度：

$v_{100}=m*v_{99}+g(w_{100})$

$=g(w_{100})+m*(m*v_{98}+g(w_{99}))$

$=g(w_{100})+m*g(w_{99})+m^2*v_{98}$

$=g(w_{100})+m*g(w_{99})+m^2*g(w_{98})+m^3*v_{97}$

可以发现当前的梯度更新量会考虑当前梯度，上一时刻的梯度，再之前时刻的梯度，这样一直往前，只是权重越来越小。

可以看下面momentum的作用：
<img style="float: center;" src="images/119.png" width="70%">
<img style="float: center;" src="images/120.png" width="70%">

可以发现动量的0.01收敛的速度变快，但是前面会有一些震荡，这是因为这里的m过大，导致当日温度的权重太小，所以前面梯度一旦大小变化，这里就会震荡，之后震荡会越来越小最后趋于平缓，这是因为不断平均的梯度越来越多。

如果减少动量m，效果会好一些：
<img style="float: center;" src="images/121.png" width="70%">

### 常用优化器

optim.SGD(params, lr=\<object object>, momentum=0, dampening=0, weight_decay=0, nesterov=False)
- param：管理的参数组
- lr：初始学习率
- momentum：动量系数，$\beta$
- weight_decay：L2正则化系数
- nesterov：是否采用NAG

同时还有10个其他的优化器：
- optim.SGD: 随机梯度下降法
- optim.Adagrad: 自适应学习率梯度下降法
- optim.RMSprop: Adagrad的改进
- optim.Adadelta: Adagrad的改进
- optim.Adam: RMSprop结合Momentum
- optim.Adamax: Adam增加学习率上限
- optim.SparseAdam: 稀疏版的Adam
- optim.ASGD: 随机平均梯度下降
- optim.Rprop: 弹性反向传播
- optim.LBFGS: BFGS的改进

这里比较常用的是：optim.SGD和optim.Adam

# 学习率调整策略

优化器中有很多超参数（学习率，动量系数等），其中最重要的一个参数是学习率，直接控制了参数更新步伐的大小，整个训练当中，学习率不是一成不变，也可以调整和优化。

## 为何要调整学习率

学习率可以控制更新的步伐，在训练模型的时候，一般开始的时候学习率会比较大，可以快速到达最优点附近，然后再把学习率降下来，缓慢取收敛达到最优值。
<img style="float: center;" src="images/122.png" width="70%">

打高尔夫的时候，一般大力把球打到洞口旁边，然后把力度降下来，一步步把球打到洞口

## PyTorch学习率调整策略

\_LRScheduler类：
```python
class _LRScheduler(object):
    def __init__(self, optimizer, last_epoch=-1):
        pass
    
    def get_lr(self):     # 虚函数
        raise NotlmplementedError
```

主要属性：
- optimizer：关联优化器，需要关联一个优化器才能改动学习率
- last_epoch：记录epoch数，学习率调整以epoch为周期
- base_lrs：记录初始学习率

主要方法：
- step()：更新下一个epoch学习率，与用户对接
- get_lr()：虚函数，计算下一个epoch学习率，更新过程中的一个步骤

以人民币二分类的例子，观察LRScheduler的构建和使用
<img style="float: center;" src="images/123.png" width="70%">

步入lr_scheduler.StepLR类（继承_LRScheduler），运行到初始化的父类初始化那一行，步入：
<img style="float: center;" src="images/124.png" width="70%">

观察父类\_\_init\_\_如何去构建一个最基本的Scheduler：
<img style="float: center;" src="images/125.png" width="70%">

这样就构建好一个Scheduler，之后调用step()方法更新学习率：
<img style="float: center;" src="images/126.png" width="70%">

之后进入\_LRScheduler的step函数：
<img style="float: center;" src="images/127.png" width="70%">

之后进入StepLR类，由于get_lr在基类中是虚函数，后面编写的Scheduler要继承这个基类，并且覆盖get_lr函数，不然程序无法得知如何衰减学习率。

可以看到这里用到了初始化时候的base_lr属性：
<img style="float: center;" src="images/128.png" width="70%">

优化器定义和使用的内部运行原理总结：

- 定义优化器时完成优化器的初始化工作：
  - 关联优化器（self.optimizer属性）
  - 初始化last_epoch和base_lr（记录原始的学习率，后续get_lr会用到）
- 用Scheduler，直接用step()方法进行更新下一个epoch的学习率（注意放在epoch的循环里而不是batch的循环里，否则学习率会变得非常小）
- 内部再_Scheduler类step()方法里调用get_lr()方法，该方法需要我们写Scheduler的时候自己覆盖，告诉程序按照什么样的方式去更新学习率
- 这样程序根据方式去计算下一个epoch的学习率，然后直接更新进优化器的_param_groups()里

### 六种学习率调整策略

- 有序调整：Step、MultiStep、 Exponential、CosineAnnealing，
  - 需要事先知道学习率大体需要在多少个epoch之后调整的时候用
- 自适应调整：ReduceLROnPleateau
  - 可以监控某个参数，根据参数的变化情况自适应调整
- 自定义调整：Lambda
  - 在模型的迁移中或者多个参数组不同学习策略的时候实用

#### StepLR

等间隔调整学习率

lr_scheduler.StepLR(optimizer, step_size, gamma=0.1, last_epoch=-1)
- step_size：调整间隔数
- gamma：调整系数，调整方式：$lr=lr*gamma$，gamma一般取0.1-0.5

使用时指定step_size，比如50，就是50个epoch调整一次学习率。
<img style="float: center;" src="images/129.png" width="70%">

#### MultiStepLR

按给定间隔调整学习率

lr_scheduler.MultiStepLR(optimizer, milestones, gamma=0.1, last_epoch=-1)
- milestones：设定调整时刻数
- gamma：调整系数，调整方式：$lr=lr*gamma$

这里可以设置调整的间隔，构建一个list，比如[50, 125, 150]，放到milestones中，则就是50个epoch，125个epoch，150个epoch调整一次学习率。

<img style="float: center;" src="images/130.png" width="70%">

#### ExponentialLR

按指数衰减调整学习率

lr_scheduler.ExponentialLR(optimizer, gamma, last_epoch=-1)

- gamma：指数的底，调整方式：$lr=lr*gamma^{epoch}$

<img style="float: center;" src="images/131.png" width="70%">

#### CosineAnnealingLR

余弦周期调整学习率

lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1)
- T_max：下降周期（只是往下的那一块）
- eta_min：学习率下限

调整方式：$\eta_t=\eta_{min}+\frac{1}{2}(\eta_{max}-\eta_{min})\left(1+\cos\left(\frac{T_{cur}}{T_{max}}\pi\right)\right)$

<img style="float: center;" src="images/132.png" width="70%">

#### ReduceLROnPlateau

监控指标，当指标不再变化则调整（非常实用），可以监控loss或准确率，当不在变化的时候，再去调整

lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10, verbose=False, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-8)

- mode：min/max两种模式
  - min：监控指标不下降调整，比如loss
  - max：监控指标不上升调整，比如acc
- factor：调整系数，类似上面的gamma
- patience：“耐心”，接受几次不变化，这一定要是连续多少次不发生变化
- cooldown：“冷却时间”，停止监控一段时间
- verbose：是否打印日志，也就是什么时候更新了我们的学习率
- min_lr：学习率下限
- eps：学习率衰减最小值

这个是学习率一直保持不变：
<img style="float: center;" src="images/133.png" width="70%">

如果在第5个epoch更新一下：
<img style="float: center;" src="images/134.png" width="70%">

#### LambdaLR

自定义调整策略（比较实用），告诉程序我们想如何改变学习率。

同时可以对不同的参数组设置不同的学习率调整方法，在模型微调中很有用。

lr_scheduler.LambdaLR(optimizer, lr_lambda, last_epoch=-1)
- lr_lambda：function或者list

<img style="float: center;" src="images/135.png" width="70%">

实现过程：

依然调用get_lr()函数，可以看看这里到底是如何实现自定义的：
<img style="float: center;" src="images/136.png" width="70%">

再次stepinto，发现跳到了我们自定义的两个更新策略上来：
<img style="float: center;" src="images/137.png" width="70%">

### 初始学习率

两种学习率初始化方式：
- 设置较小值：0.01，0.001，0.0001
- 搜索最大学习率：论文《Cyclical Learning Rates for Training Neural Networks》，先让学习率从0开始慢慢增大，然后观察acc，找到训练准确率下降的点作为初始学习率

<img style="float: center;" src="images/138.png" width="70%">

# 思维导图

<img style="float: center;" src="images/139.png" width="90%">